<a href="https://colab.research.google.com/github/AdityaTanejaRox/SpamEmailClassification/blob/main/Bayes_Classifier_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**#NOTEBOOK IMPORTS**

In [ ]:
import pandas as pd
import numpy as np

**#CONSTANTS**

In [ ]:
TRAINING_DATA_FILE = 'SpamData/02_Training/train-data.txt'
TEST_DATA_FILE = 'SpamData/02_Training/test-data.txt'

TOKEN_SPAM_PROB_FILE = 'SpamData/03_Testing/prob-spam.txt'
TOKEN_HAM_PROB_FILE = 'SpamData/03_Testing/prob-ham.txt'
TOKEN_ALL_PROB_FILE = 'SpamData/03_Testing/prob-all-tokens.txt'

TEST_FEARURE_MATRIX = 'SpamData/03_Testing/test-features.txt'
TEST_TARGET_FILE = 'SpamData/03_Testing/test-target.txt'

VOCAB_SIZE = 2500

**#READ AND LOAD FEATURES FROM TXT FILES INTO NP ARRAY**

In [ ]:
sparse_train_data = np.loadtxt(TRAINING_DATA_FILE, delimiter=' ', dtype=int)

In [ ]:
sparse_test_data = np.loadtxt(TEST_DATA_FILE, delimiter=' ', dtype=int)

In [ ]:
sparse_train_data[:5]

array([[ 0,  2,  1,  1],
       [ 0,  3,  1,  2],
       [ 0,  4,  1,  1],
       [ 0,  7,  1,  3],
       [ 0, 11,  1,  1]])

In [ ]:
sparse_train_data[-5:]

array([[5798, 2030,    0,    1],
       [5798, 2033,    0,    2],
       [5798, 2278,    0,    1],
       [5798, 2342,    0,    1],
       [5798, 2372,    0,    1]])

In [ ]:
print('No.of rows in training file:', sparse_train_data.shape[0])
print('No.of rows in testing file:', sparse_test_data.shape[0])

No.of rows in training file: 262774
No.of rows in testing file: 111952


In [ ]:
print('No.of Emails in training file:', np.unique(sparse_train_data[:, 0]).size)#selecr all the rows in 1st column

No.of Emails in training file: 4012


In [ ]:
print('No.of Emails in testing file:', np.unique(sparse_test_data[:, 0]).size)#selecr all the rows in 1st column

No.of Emails in testing file: 1725


**CREATING AN EMPTY DATAFRAME**

In [ ]:
column_names = ['DOC_ID'] + ['CATEGORY'] + list(range(0, VOCAB_SIZE))
column_names[:5]

['DOC_ID', 'CATEGORY', 0, 1, 2]

In [ ]:
index_names = np.unique(sparse_train_data[:, 0])
index_names

array([   0,    1,    2, ..., 5796, 5797, 5798])

In [ ]:
full_train_data = pd.DataFrame(index=index_names, columns=column_names)
full_train_data.fillna(value=0, inplace=True)

In [ ]:
full_train_data.head()

,DOC_ID,CATEGORY,0,1,2,3,4,5,6,7,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
sparse_train_data[10][3]

1

**#CREATE FULL MATRIX FROM SPARSE MATRIX**

In [ ]:
def make_full_matrix(sparse_matrix, nr_words, doc_idx=0, word_idx=1, cat_idx=2, freq_idx=3):
    """Form a full matrix from sparse matrix.
        Returns a pandas dataframe.
        Keyword Arguements:
        sparse_matrix -- numpy array.
        nr_words -- size of vocabulary. Total No.of tokens.
        doc_idx -- position of document in the sparse_matrix. Default:1st column.
        word_idx -- position of the word in the sparse_matrix. Default:2nd column.
        cat_idx -- position of the label (1:spam, 2:nonspam). Default:3rd column.
        freq_idx -- position of occurrence of word in sparse_matrix. Default:4th column."""
  
    column_names = ['DOC_ID'] + ['CATEGORY'] + list(range(0, VOCAB_SIZE))
    doc_id_names = np.unique(sparse_matrix[:, 0])
    full_matrix = pd.DataFrame(index=doc_id_names, columns=column_names)
    full_matrix.fillna(value=0, inplace=True)
 
    for i in range(sparse_matrix.shape[0]):
        doc_nr = sparse_matrix[i][doc_idx]
        word_id = sparse_matrix[i][word_idx]
        label = sparse_matrix[i][cat_idx]
        occurrence = sparse_matrix[i][freq_idx]
        
        full_matrix.at[doc_nr, 'DOC_ID'] = doc_nr
        full_matrix.at[doc_nr, 'CATEGORY'] = label
        full_matrix.at[doc_nr, word_id] = occurrence
        
    full_matrix.set_index('DOC_ID', inplace=True)    
    return full_matrix

In [ ]:
%%time
full_train_data = make_full_matrix(sparse_train_data, VOCAB_SIZE)

Wall time: 38.2 s


In [ ]:
full_train_data.head()

,CATEGORY,0,1,2,3,4,5,6,7,8,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,1,0,0,1,2,1,0,0,3,0,...,0,0,0,0,0,0,0,0,0,0
1,1,7,1,2,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,1,6,1,1,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,1,6,0,0,2,4,0,3,14,0,...,0,0,0,0,0,0,0,0,0,0
4,1,5,1,2,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
full_train_data.tail()

,CATEGORY,0,1,2,3,4,5,6,7,8,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
5792,0,3,1,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
5795,0,2,2,0,1,0,0,2,1,0,...,0,0,0,0,0,0,0,0,0,0
5796,0,1,9,1,0,0,1,2,1,1,...,0,0,0,0,0,0,0,0,0,0
5797,0,1,1,1,0,0,1,2,0,0,...,0,0,0,0,0,0,0,0,0,0
5798,0,3,4,2,0,5,0,3,0,0,...,0,0,0,0,0,0,0,0,0,0


**#TRAINING THE NAIVE BAYES MODEL**

**Calculating the Probability of Spam**


In [ ]:
#Calculate the Probability of Spam - the % of spam messages in the training dataset.
full_train_data.CATEGORY.size

4012

In [ ]:
full_train_data.CATEGORY.sum()

1246

In [ ]:
prob_spam = full_train_data.CATEGORY.sum() / full_train_data.CATEGORY.size
print(f'Probability of spam is:', prob_spam)

Probability of spam is: 0.31056829511465606


**TOTAL NO.OF WORDS & TOKENS**

In [ ]:
full_train_features = full_train_data.loc[:, full_train_data.columns != 'CATEGORY']
full_train_features.head()

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,0,0,1,2,1,0,0,3,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,1,1,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,6,0,0,2,4,0,3,14,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
email_lengths = full_train_features.sum(axis=1)
email_lengths.shape

(4012,)

In [ ]:
email_lengths[:5]

DOC_ID
0     87
1     53
2     40
3    183
4     43
dtype: int64

In [ ]:
total_wc = email_lengths.sum()
total_wc

435983

**NO.OF TOKENS IN SPAM AND HAM EMAILS**

In [ ]:
spam_lengths = email_lengths[full_train_data.CATEGORY == 1]
spam_wc = spam_lengths.sum()
print('Word count of spam emails:', spam_wc)
ham_lengths = email_lengths[full_train_data.CATEGORY == 0]
ham_wc = ham_lengths.sum()
print('Word count of ham emails:', ham_wc)

Word count of spam emails: 189463
Word count of ham emails: 246520


In [ ]:
email_lengths.shape[0] - spam_lengths.shape[0] - ham_lengths.shape[0]

0

In [ ]:
print('Average No.of Words in spam emails {:.0f}'.format(spam_wc / spam_lengths.shape[0]))
print('Average No.of Words in ham emails {:.0f}'.format(ham_wc / ham_lengths.shape[0]))

Average No.of Words in spam emails 152
Average No.of Words in ham emails 89


**##SUMMING THE TOKENS OCCURING IN SPAM**

In [ ]:
full_train_features.shape

(4012, 2500)

In [ ]:
train_spam_tokens = full_train_features.loc[full_train_data.CATEGORY == 1]
train_spam_tokens.head()

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,0,0,1,2,1,0,0,3,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,1,1,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,6,0,0,2,4,0,3,14,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
train_spam_tokens.tail()

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
1887,0,0,0,0,0,0,0,2,2,0,...,0,0,0,0,0,0,0,0,0,0
1889,0,0,2,3,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1890,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1895,0,3,1,1,1,1,44,1,5,5,...,0,0,0,0,0,0,0,0,0,0
1896,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [ ]:
train_spam_tokens.shape

(1246, 2500)

In [ ]:
summed_spam_tokens = train_spam_tokens.sum(axis=0) + 1 ##adding 1 coz if a token doesn't
##occur then we will be dividing 0/no.of tokens. we are performing LAPLACE smoothing.
summed_spam_tokens.shape

(2500,)

**##SUMMING HAM TOKENS**

In [ ]:
train_ham_tokens = full_train_features.loc[full_train_data.CATEGORY == 0]
train_ham_tokens.head()
summed_ham_tokens = train_ham_tokens.sum(axis=0) + 1

In [ ]:
summed_ham_tokens.shape

(2500,)

In [ ]:
summed_ham_tokens.head()

0    5317
1    2573
2    2052
3     892
4    1656
dtype: int64

**##P(Token | Spam) - Probability that a Token occurs given Email is Spam.**

In [ ]:
prob_tokens_spam = summed_spam_tokens / (spam_wc + VOCAB_SIZE) ##adding to implement LAPLACE SMOOTHING
prob_tokens_spam

0       0.011507
1       0.005136
2       0.007111
3       0.010658
4       0.007178
          ...   
2495    0.000031
2496    0.000094
2497    0.000010
2498    0.000083
2499    0.000151
Length: 2500, dtype: float64

In [ ]:
prob_tokens_spam.sum()

1.0

**##P(Token | Ham) - Probability that a Token occurs given Email is Ham.**

In [ ]:
prob_tokens_nonspam = summed_ham_tokens / (ham_wc + VOCAB_SIZE)
prob_tokens_nonspam[:5]

0    0.021352
1    0.010333
2    0.008240
3    0.003582
4    0.006650
dtype: float64

In [ ]:
prob_tokens_nonspam.sum()

1.0

**##P(Token) - Probability that token occurs**

In [ ]:
prob_tokens_all = full_train_features.sum(axis = 0) / total_wc
prob_tokens_all.sum()

1.0

**##SAVE THE TRAINED MODEL**

In [ ]:
np.savetxt(TOKEN_SPAM_PROB_FILE, prob_tokens_spam)
np.savetxt(TOKEN_HAM_PROB_FILE, prob_tokens_nonspam)
np.savetxt(TOKEN_ALL_PROB_FILE, prob_tokens_all)

**##PREPARE THE TEST DATA**

In [ ]:
sparse_test_data.shape

(111952, 4)

In [ ]:
%%time
full_test_data = make_full_matrix(sparse_test_data, nr_words = VOCAB_SIZE)

Wall time: 17.9 s


In [ ]:
X_test = full_test_data.loc[:, full_test_data.columns != 'CATEGORY']
y_test = full_test_data.CATEGORY

In [ ]:
np.savetxt(TEST_TARGET_FILE, y_test)
np.savetxt(TEST_FEARURE_MATRIX, X_test)